In [ ]:
!git clone https://github.com/yeaung276/FaceRecognization.git

In [ ]:
!pip install tfx

In [ ]:
!cd FaceRecognization/pipeline

In [ ]:
!pwd

/content/FaceRecognization


In [ ]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from pipeline.example_gen import TripletExampleGen
from tfx.proto import example_gen_pb2

context = InteractiveContext()
example_gen = TripletExampleGen(
    input_base='mocks/example_gen',
    input_config=example_gen_pb2.Input(splits=[
          example_gen_pb2.Input.Split(name='train', pattern='[0-2]'),
          example_gen_pb2.Input.Split(name='eval', pattern='[3-4]')
          ])
    )
context.run(example_gen)

ExecutionResult(
    component_id: TripletExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=TripletExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [ ]:
from tfx.dsl.components.common.importer import Importer
from tfx.types import standard_artifacts

model = Importer(
    source_uri='models/mobile_net',
    artifact_type=standard_artifacts.Model
  ).with_id('model_importer')

context.run(model)

ExecutionResult(
    component_id: model_importer
    execution_id: 2
    outputs:
        result: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "models/mobile_net"
        custom_properties {
          key: "is_external"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.14.0"
          }
        }
        , artifact_type: id: 16
        name: "Model"
        base_type: MODEL
        )]
            additional_properties: {}
            additional_custom_properties: {}
            _input_trigger: None
        ))

In [ ]:
from pipeline.embedding_gen import EmbeddingGen

encoding_gen = EmbeddingGen(
      examples=example_gen.outputs['examples'],
      model=model.outputs['result']
    )
context.run(encoding_gen)

ExecutionResult(
    component_id: EmbeddingGen
    execution_id: 3
    outputs:
        output: OutputChannel(artifact_type=Examples, producer_component_id=EmbeddingGen, output_key=output, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [ ]:
!python main.py

2023-11-02 17:10:45.186846: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using TensorFlow backend
Traceback (most recent call last):
  File "/content/FaceRecognization/main.py", line 6, in <module>
    pipeline = create_pipeline('test', 'mocks/pipeline_root', 'mocks/example_gen', 'mocks/pipeline_metadata')
  File "/content/FaceRecognization/pipeline/pipeline.py", line 35, in create_pipeline
    encoding_gen = EmbeddingGen(
  File "/content/FaceRecognization/pipeline/embedding_gen/component.py", line 32, in __init__
    examples_artifact.split_names = examples.get()[0].split_names  # type: ignore
IndexError: list index out of range
